# Fase 1: Extracción de Datos de API de MoviesDataset

API: https://rapidapi.com/SAdrian/api/moviesdatabase

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [2]:
#creamos el diccionario donde alamcenaremos la información
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[],
                 'Genero':[]}

#Creamos varias listas para guardar la información de cada key del diccionario que nos devuelve la función. 
lista_tipos = []
lista_nombre = []
lista_anno = []
lista_mes = []
lista_id = []
lista_generos = []


# creamos una lista para almacenar las respuestas de la funcion de llamada a la API por si las queremos consultar
lista_respuestas = []
lista_info_respuestas = []


#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

# API_key Nuria (no funciona) = '505782c7fbmsh5a469e43b5fecf5p138dfajsna7d8efc4677f'
# API_key Sharon = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
# API_key3 Silvia = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
# API_key3 Fer (no funciona) = 'd215e5295fmsh4ea493c17eb16a6p1ad050jsnd6897569237c'
# API_key Isa1 (no funciona) = '555502744dmsh2f20b5fd2ccbb41p11ac03jsndaa25d7ef539'
# API_key Isa2 (no funciona)= '7a833b5f94msh046ef16d4b5eb88p18a187jsn006b25582aff'

# headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}

titleType_list = ['movie', 'short']
genere_list = ['Action', 'Drama', 'Comedy']

# querystring = {"titleType":"short","startYear":"2010","endYear":"2024","genre":"Drama"}


In [3]:
# Definimos una fucnción para extraer información de la API y almacenarla en un diccionario

def llamar_API (url,type, genere, **dict):
    '''
    Esta función realiza una llamada a una API a partir de una url y guarda la información en un diccionario.

    Parametros:
    -----------
    url (str): la dirección url de la API a la que queremos hacer la llamada
    type (str): El tipo de pelícua que queremos buscar. 
    year (str): El año de la película que queremos buscar.
    genere (dtr): El género que ueremos buscar. 
    dict (dict): el diccionario dónde se almacenará la información

    Returns:
    -------
    dict (dict): el diccionario donde se almacenará la información
    '''
    #definimos la querystring y la API key
    querystring = {"titleType":type,"startYear":"2010","endYear":"2024","genre":genere}

    # API_key = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
    API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
        

    headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}    

    #iniciamos el loop while para navegar por las distintas páginas
    while url.startswith('https'):

        #realizmos la llamada y almacenamos los códigos de respuesta en la lista de respuestas
        response_pelis = requests.get(url, headers=headers, params=querystring)
        lista_respuestas.append(response_pelis.status_code)
        lista_info_respuestas.append(response_pelis.reason)

        if response_pelis.status_code == 200:

            #convertimos los resulatos a formato json
            js_response_pelis = response_pelis.json()

            #guardamos la inforación de la API dónde tenemos la dirección de la siguiente página
            next = js_response_pelis['next']

            #guardamos la parte donde tenemos la infomación en una variable
            info_pelis = js_response_pelis['results']

            #guardamos toda la información en el diccionario con un for loop    
            for peli in info_pelis:

                dict['Genero'].append(genere)
                dict['Tipo'].append(peli['titleType']['text'])
                dict['Nombre'].append(peli['titleText']['text'])
                dict['Id'].append(peli['id'])
                dict['Año_estreno'].append(peli['releaseYear']['year'])

                #Hacemos un try/expect para las películas que no se han estrenado todavía            
                try:
                    dict['Mes_estreno'].append(peli['releaseDate']['month'])
                except:            
                    dict['Mes_estreno'].append('por estrenar')

            #iniciamos el loop try-except para que cree la variable de la nueva url mientras tengamos una url en ´next´
            try:
                #definimos la base de la nueva url
                url_fixed = "https://moviesdatabase.p.rapidapi.com"
                #definimos la nueva url
                url=url_fixed+next
                #definimos el nuevo querystring
                querystring = {}

            except:
                break

        else:
            print(f'Error {response_pelis.status_code} en la llamada a la API')
            break


    #mostramos los resultados
    if response_pelis.status_code == 200:
        print(f'Fin: Se han añadido l@s {type}s del género {genere} a tu diccionario.')

In [5]:
# Iniciamos el loop for para conseguir información de todos los tipos y años del género Action
# genere = 'Action'

#Iteramos por cada año, desde el 2010 hasta el 2025.
# for year in range(2010,2025):
#     print('Año:', year)
# year = 2010
# print('Año:', year)

for genere in genere_list:

    #Iteramos por cada tipo: Shot and Movie
    for type in titleType_list:

        #iniciamos la funcion que llama a la API     
        llamar_API(url,type, genere, **dic_API_Pelis)

#Añadimos la información en las listas con un extend
lista_tipos.extend(dic_API_Pelis['Tipo'])
lista_nombre.extend(dic_API_Pelis['Nombre'])
lista_anno.extend(dic_API_Pelis['Año_estreno'])
lista_mes.extend(dic_API_Pelis['Mes_estreno'])
lista_id.extend(dic_API_Pelis['Id'])
lista_generos.extend(dic_API_Pelis['Genero'])

Fin: Se han añadido l@s movies del género Action a tu diccionario.
Fin: Se han añadido l@s shorts del género Action a tu diccionario.
Fin: Se han añadido l@s movies del género Drama a tu diccionario.
Fin: Se han añadido l@s shorts del género Drama a tu diccionario.
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.


In [6]:
len(lista_id)

7420

In [ ]:
# # Iniciamos el loop for para conseguir información de todos los tipos y años del género Action
# genere = 'Drama'

# #Iteramos por cada año, desde el 2010 hasta el 2025.
# for year in range(2010,2025):
#     print('Año:', year)

#     #Iteramos por cada tipo: Shot and Movie
#     for type in titleType_list:

#         #iniciamos la funcion que llama a la API     
#         llamar_API(url,type,year, genere, **dic_API_Pelis)


# #Añadimos la información en las listas con un extend
# lista_tipos.extend(dic_API_Pelis['Tipo'])
# lista_nombre.extend(dic_API_Pelis['Nombre'])
# lista_anno.extend(dic_API_Pelis['Año_estreno'])
# lista_mes.extend(dic_API_Pelis['Mes_estreno'])
# lista_id.extend(dic_API_Pelis['Id'])
# lista_generos.extend(dic_API_Pelis['Genero'])

In [ ]:
# # Iniciamos el loop for para conseguir información de todos los tipos y años del género Action
# genere = 'Comedy'

# #Iteramos por cada año, desde el 2010 hasta el 2025.
# for year in range(2010,2025):
#     print('Año:', year)

#     #Iteramos por cada tipo: Shot and Movie
#     for type in titleType_list:

#         #iniciamos la funcion que llama a la API     
#         llamar_API(url,type,year, genere, **dic_API_Pelis)


# #Añadimos la información en las listas con un extend
# lista_tipos.extend(dic_API_Pelis['Tipo'])
# lista_nombre.extend(dic_API_Pelis['Nombre'])
# lista_anno.extend(dic_API_Pelis['Año_estreno'])
# lista_mes.extend(dic_API_Pelis['Mes_estreno'])
# lista_id.extend(dic_API_Pelis['Id'])
# lista_generos.extend(dic_API_Pelis['Genero'])

In [7]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
zip_fase1 = zip(lista_tipos, lista_nombre, lista_anno, lista_mes, lista_id, lista_generos)
lista_tuplas_fase1 = list(zip_fase1)
len(lista_tuplas_fase1)

7420

In [15]:
# Convertimos la lista de tuplas con la informacion a Data Frame y borramos los duplicados
df_fase1=pd.DataFrame(lista_tuplas_fase1)
df_fase1.columns = ['Tipo', 'Nombre', 'Anno_estreno', 'Mes_estreno', 'Id_peli','Genero']
df_fase1_final = df_fase1.drop_duplicates(subset='Id_peli')


In [17]:
# Convertimos el DataFrame a formato .csv
df_fase1_final.to_csv("Fase1_2010_to_2024.csv", index=False)

In [16]:
df_fase1_final['Id_peli'].duplicated().sum()

0

In [64]:
df_fase1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53008 entries, 0 to 53007
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tipo          53008 non-null  object
 1   Nombre        53008 non-null  object
 2   Anno_estreno  53008 non-null  int64 
 3   Mes_estreno   50474 non-null  object
 4   Id_peli       53008 non-null  object
 5   Genero        53008 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


In [ ]:
# Convertimos la lista de tuplas con la informacion a Data Frame y luego lo exportamos con un fichero formato .csv
df2_fase1=pd.DataFrame(lista_tuplas_fase1)
df2_fase1.columns = ['Tipo', 'Nombre', 'Anno_estreno', 'Mes_estreno', 'Id_peli','Genero']
df2_fase1.to_csv("Fase1.csv", index=False)